In [18]:
from torchvision.transforms.transforms import ToTensor
import torch 
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_ds= torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_ds= torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

train_dl = torch.utils.data.DataLoader(dataset=train_ds, batch_size=128, shuffle=True)
test_dl = torch.utils.data.DataLoader(dataset=test_ds, batch_size=128, shuffle=False)

examples = iter(test_dl)
example_images, example_targets = examples.next()

# for i in range(8):
#   plt.subplot(2,4,i+1)
#   plt.imshow(example_images[i][0], cmap='gray')
# plt.show()

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.fc1 = nn.Linear(28 * 28, 100)
    self.fc2 = nn.Linear(100, 100)
    self.fc3 = nn.Linear(100, 10)
    self.dropout = nn.Dropout(0.2)

  def forward(self, x):
    x = x.view(-1, 28 * 28)
    x = torch.relu(self.fc1(x))
    x = self.dropout(x)
    x = torch.relu(self.fc2(x))
    x = self.dropout(x)
    x = self.fc3(x)
    return x

model = NeuralNetwork().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001, betas=[0.9, 0.999])

n_epochs = 10

model.train()

for epoch in range(n_epochs):
   
    train_loss = 0.0
    
    for data, target in train_dl:
       
        optimizer.zero_grad()
        
        output = model(data)
        
        loss = criterion(output, target)
       
        loss.backward()
        
        optimizer.step()
       
        train_loss += loss.item()*data.size(0)
             
    train_loss = train_loss/len(train_dl.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss
        ))
    
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()

for data, target in test_dl:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # getting the current batch size
    batch_size = data.size(0)
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(test_dl.dataset)
print('\nTest Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i+1), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))


  





Epoch: 1 	Training Loss: 0.494300
Epoch: 2 	Training Loss: 0.213224
Epoch: 3 	Training Loss: 0.161155
Epoch: 4 	Training Loss: 0.133939
Epoch: 5 	Training Loss: 0.116759
Epoch: 6 	Training Loss: 0.101397
Epoch: 7 	Training Loss: 0.092548
Epoch: 8 	Training Loss: 0.084792
Epoch: 9 	Training Loss: 0.077437
Epoch: 10 	Training Loss: 0.072082

Test Loss: 0.075522

Test Accuracy of     1: 99% (971/980)
Test Accuracy of     2: 98% (1118/1135)
Test Accuracy of     3: 97% (1008/1032)
Test Accuracy of     4: 97% (986/1010)
Test Accuracy of     5: 98% (963/982)
Test Accuracy of     6: 96% (857/892)
Test Accuracy of     7: 98% (939/958)
Test Accuracy of     8: 97% (1004/1028)
Test Accuracy of     9: 98% (956/974)
Test Accuracy of    10: 96% (970/1009)

Test Accuracy (Overall): 97% (9772/10000)
